In [1]:
import numpy as np
import pandas as pd
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from datasets import Dataset, load_dataset
from numpy import genfromtxt
from logging import INFO
import xgboost as xgb
from flwr.common import log
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import IidPartitioner
from pandas import concat

/home/zuy/anaconda3/envs/flower_tutorial/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-22 10:20:34,714	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
df = pd.read_csv('/home/zuy/Documents/BCU/ML/CSV/df_train_3.csv')
    df.drop("Unnamed: 0", axis=1, inplace=True)

In [5]:
df.shape

(227845, 18)

In [38]:
df.Class.value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [7]:
"""Load partition HIGGS data."""
# Only initialize `FederatedDataset` once
fds = None  # Cache FederatedDataset
if fds is None:
    partitioner = IidPartitioner(num_partitions=1)
    fds = FederatedDataset(
        dataset="jxie/higgs",
        partitioners={"train": partitioner},
    )
    partition = fds.load_partition(0, split="train")
    partition.set_format("numpy")

Generating val_16k split: 100%|██████████| 15688/15688 [00:00<00:00, 407005.93 examples/s]


In [ ]:
"""Load partition HIGGS data."""
# Only initialize `FederatedDataset` once
fds = None  # Cache FederatedDataset
if fds is None:
    partitioner = IidPartitioner(num_partitions=1)
    fds = FederatedDataset(
        dataset="ylecun/mnist",
        partitioners={"train": partitioner},
    )
    
# Load the partition for this `partition_id`
partition = fds.load_partition(0, split="train").set_format("numpy")
# Train/test splitting
train_data, valid_data, num_train, num_val = train_test_num_split(partition, test_fraction=0.2, seed=42)

In [ ]:
"""Load partition df_3 data."""
df = pd.read_csv('/home/zuy/Documents/BCU/ML/CSV/df_3.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)
dataset = Dataset.from_pandas(df)
partitioner = IidPartitioner(num_partitions=1)
partitioner.dataset = dataset
dataset = partitioner.load_partition(partition_id=0).to_pandas()

X = dataset.drop('Class', axis=1)
y = dataset['Class']
# new_data = xgb.DMatrix(X, label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.2, random_state=42, stratify=y)
partition_train = pd.concat([X_train, X_test])
partition_test = pd.concat([y_train, y_test])
num_train = len(partition_train)
num_test = len(partition_test)


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [13]:
df = pd.read_csv('/home/zuy/Documents/BCU/ML/CSV/df_train_3.csv')
df.drop("Unnamed: 0", axis=1, inplace=True)

In [14]:
df.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Amount,Class
0,0.905774,0.862468,-0.582668,-0.800214,-0.402839,1.195050,3.686711,-1.352372,2.084633,1.176746,-0.064710,-0.054899,0.360069,0.057693,-0.126514,-0.204315,0.0
1,0.465984,0.902013,-0.081009,-1.688308,0.169440,2.300397,3.063923,-0.037911,1.216367,0.077896,0.381968,-0.149059,0.179281,-0.298349,0.731565,-0.264579,0.0


In [16]:
df_new = MinMaxScaler().fit_transform(df)
df_new = pd.DataFrame(df_new, columns=[
'Time','V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','Amount','Class'])

In [17]:
df_new.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,Amount,Class
0,0.937075,0.991418,0.759318,0.813931,0.222601,0.775833,0.303487,0.256341,0.797613,0.511242,0.505465,0.278523,0.723425,0.453073,0.643815,0.000285,0.0
1,0.720386,0.992920,0.766743,0.784411,0.262921,0.785532,0.296185,0.265342,0.792624,0.464359,0.514616,0.270128,0.716463,0.416956,0.670298,0.000116,0.0


In [21]:
df_new['Class'].unique()

array([0., 1.])